<a href="https://colab.research.google.com/github/hmaharaja/AI_stuff/blob/Steel-defect-classification/Stage1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup and Architecture

Here we will find the following sections: 
   1. Import Section
   2. Defining the training/accuracy functions 
   3. Dataset Generation
   4. Training Stage 1 AlexNet model
   5. Testing

#FORMATTING NOTES:

1. **Input size for images is:** 224 by 224
2. All images should be in greyscale (With channel dimension included)

In [0]:
#Import Section

import numpy as np
np.set_printoptions(suppress=True)
import scipy.signal as sg
from PIL import Image, ImageDraw
import requests
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt # for plotting
import torch.optim as optim #for gradient descent
import torchvision
import torch.utils.data as data
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms, utils, models

import csv
import seaborn as sn
import math
import os
import shutil
import random
import cv2
from scipy import ndimage, misc
import itertools
from skimage import io, transform
from pathlib import Path

#20 Samples Import


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


#Define Training and Accuracy Functions

In [0]:
def get_accuracy(model, data_loader, useGPU=True):
    correct = 0
    total = 0
    for imgs, labels in data_loader:
        #To Enable GPU Usage
        if useGPU and torch.cuda.is_available():
            imgs = imgs.cuda()
            labels = labels.cuda()
            model = model.cuda()
        output = model(imgs)
        
        #select index with maximum prediction score
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += imgs.shape[0]
        
    return correct / total

In [0]:
def train(model, train_data, val_data, batch_size=64, num_epochs=1, learning_rate=0.01, useGPU=True):
    
    #Put data in data loaders
    train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
                                           num_workers=0, shuffle=False)
    val_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
                                           num_workers=0, shuffle=False)
   
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
    #optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    iters, losses, train_acc, val_acc = [], [], [], []

    if useGPU and torch.cuda.is_available():
        model = model.cuda()
        print("Training on GPU")
   
    # training
    n = 0 # the number of iterations
    for epoch in range(num_epochs):
        for imgs, labels in iter(train_data_loader):
          #To Enable GPU Usage
          if useGPU and torch.cuda.is_available():
              imgs = imgs.cuda()
              labels = labels.cuda()
          
          out = model(imgs)             # forward pass

          loss = criterion(out, labels) # compute the total loss
          loss.backward()               # backward pass (compute parameter updates)
          optimizer.step()              # make the updates for each parameter
          optimizer.zero_grad()         # a clean up step for PyTorch

          # save the current training information
          iters.append(n)
          losses.append(float(loss)/batch_size)             # compute *average* loss
          train_acc.append(get_accuracy(model, train_data_loader, useGPU)) # compute training accuracy 
          val_acc.append(get_accuracy(model, val_data_loader, useGPU))  # compute validation accuracy
          print("Iteration: ", str(n), "| Train Loss: ", losses[n], "| Train Accuracy: ", train_acc[n], "| Validation Accuracy: ", val_acc[n])
          n += 1
        #print(("Epoch {}: Train loss: {}, "+"Train accuracy: {}"+"| "+"Validation accuracy: {}").format(epoch + 1,losses[epoch],
                                                                                                        #train_acc[epoch], val_acc[epoch]))
            
        # Save the current model (checkpoint) to a file
        model_path = "model_{0}_bs{1}_lr{2}_epoch{3}".format(model.name,
                                                    batch_size,
                                                    str(learning_rate).replace('.', '-'),
                                                    epoch)
        torch.save(model.state_dict(), model_path + ".pth")         

    # plotting
    plt.title("Training Curve")
    plt.plot(iters, losses, label="Train")
    plt.xlabel("Iterations")
    plt.ylabel("Loss")
    plt.show()

    plt.title("Training Curve")
    plt.plot(iters, train_acc, label="Train")
    plt.plot(iters, val_acc, label="Validation")
    plt.xlabel("Iterations")
    plt.ylabel("Training Accuracy")
    plt.legend(loc='best')
    plt.show()

    print("Final Training Accuracy: {}".format(train_acc[-1]))
    print("Final Validation Accuracy: {}".format(val_acc[-1]))

# Split train Data into 1 and 0

In [0]:
#Link to competition: https://www.kaggle.com/c/severstal-steel-defect-detection/overview
#Upload the Kaggle JSON file (download it from the drive to your computer first)
from google.colab import files
uploaded = files.upload()

MessageError: ignored

In [0]:
#Mount the kaggle.json file
!ls -lha kaggle.json
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
#Extract required files
!kaggle competitions download -c severstal-steel-defect-detection -q
!unzip -q train.csv.zip
!unzip -q train_images.zip -d train_images

In [0]:
#Load the dataset and list of filenames

def removeDuplicates(lst):
    #Remove duplicate values from a list
    return list(dict.fromkeys(lst))

#Returns the run-length encoded pixels for a filename and given classID from 1 to 4
def getEncodedPixels(filename, classID):
    return data[filenameAndClassIndex[filename + "_" + str(classID)]][1]

#Load train.csv as a list
trainingDataPath = "/content/train.csv"
with open(trainingDataPath, 'r') as file:
    data = list(csv.reader(file, delimiter=","))

filenameAndClassIndex = {} #dictionary to index data array based on first column's value (ImageId_ClassId)   
filenames = []

for i in range(1, len(data)):
    filenameAndClassIndex[data[i][0]] = i
    filenames.append(data[i][0].split("_")[0])
filenames = removeDuplicates(filenames) # remove duplicates from the list of filenames

In [0]:

def getOneClassFilenames(classID, filenamesByClass):
    #gets a list of filenames that are only in one class and not in multiple classes
    for i in range(5):
        others = set()
        if i != classID:
            #Get a union of all the image names not in the class
            others = others.union(set(filenamesByClass[i]))
    return list(set(filenamesByClass[classID]).difference(others)) #only keep the filenames not in others
def getImage(filename):
    #returns a PIL Image object of the image with name 'filename'
    os.chdir("/content/train_images")
    picture = Image.open(filename)
    return picture

In [0]:
#Get the number of images with defects in each class

#numInClassCounter[0] is the number of images w/ no defects, numInClassCounter[1] is the number of images w/ class 1 defects... 
#filenamesByClass is a 2D list. The first dimension corresponds to the classID (0: No defects, ..., 1: Class 1, 4: Class 4)

numClasses = 5
numInClassCounter = [0, 0, 0, 0, 0]
filenamesByClass = [[], [], [], [], []]
for img_file in filenames:
    noDefects = True
    for classID in range(1,5):
        if getEncodedPixels(img_file, classID) != '': 
              numInClassCounter[classID] += 1 
              filenamesByClass[classID].append(img_file)
              noDefects = False
    if noDefects:
        numInClassCounter[0] +=1
        filenamesByClass[0].append(img_file)

In [0]:
#Get a list of filenames which only have 1 class of defect. First dimension is the class, second dimension is the list.
#These files will be used to create the histogram
uniqueFilenamesByClass = []
for i in range(5):
    uniqueFilenamesByClass.append(getOneClassFilenames(i, filenamesByClass))

In [0]:
os.makedirs('content/split/train/')
os.makedirs('content/split/train/0')
os.makedirs('content/split/train/1')
os.makedirs('content/split/validation/0')
os.makedirs('content/split/validation/1')
os.makedirs('content/split/test/')
os.makedirs('content/split/test/0')
os.makedirs('content/split/test/1')

In [0]:
changeDir= 'content/split/train/0/'
trainDir='/content/train_images/'
random.seed(1)
train0ImageNames, val0ImageNames, test0ImageNames = [], [], []
random.shuffle(filenamesByClass[0])

train0ImageNames=filenamesByClass[0][:int(0.7*len(filenamesByClass[0]))]
val0ImageNames=filenamesByClass[0][int(0.7*len(filenamesByClass[0])):int(0.85*len(filenamesByClass[0]))]
test0ImageNames=filenamesByClass[0][int(0.85*len(filenamesByClass[0])):len(filenamesByClass[0])]
# put into train folder
for i in train0ImageNames:
  imgDir=trainDir+i
  imgchangeDir=changeDir + i
  os.rename(imgDir, imgchangeDir)
# put into validation folder
changeDir= 'content/split/validation/0/'
for i in val0ImageNames:
  imgDir=trainDir+i
  imgchangeDir=changeDir + i
  os.rename(imgDir, imgchangeDir)
# test folder
changeDir= 'content/split/test/0/'
for i in test0ImageNames:
  imgDir=trainDir+i
  imgchangeDir=changeDir + i
  os.rename(imgDir, imgchangeDir)

In [0]:
#Split into training, validation, and testing set

#First split up the filenames randomly. Use a 70-15-15 split within each class, append to the set, and then remove duplicates
trainImageNames, valImageNames, testImageNames = [], [], []
for i in range(1, numClasses): #1, numClasses if ignoring classes without defects
    filesForClass = filenamesByClass[i]
    random.seed(1)
    random.shuffle(filesForClass)

    trainImageNames.append(filesForClass[:int(0.7*len(filesForClass))])
    valImageNames.append(filesForClass[int(0.7*len(filesForClass)):int(0.85*len(filesForClass))])
    testImageNames.append(filesForClass[int(0.85*len(filesForClass)):len(filesForClass)])

#Flatten 2D lists to 1D
trainImageNames = list(itertools.chain(*trainImageNames))
valImageNames = list(itertools.chain(*valImageNames))
testImageNames = list(itertools.chain(*testImageNames))

#Remove duplicate names
trainImageNames = removeDuplicates(trainImageNames)
valImageNames = removeDuplicates(valImageNames)
testImageNames = removeDuplicates(testImageNames)

#Shuffle again to mix up the classes
random.shuffle(trainImageNames)
random.shuffle(valImageNames)
random.shuffle(testImageNames)

In [0]:
changeDir= 'content/split/train/1/'
trainDir='/content/train_images/'

# put into train folder
for i in train0ImageNames:
  imgDir=trainDir+i
  imgchangeDir=changeDir + i
  my_file = Path(imgDir)
  if my_file.is_file():
      os.rename(imgDir, imgchangeDir)
# put into validation folder
changeDir= 'content/split/validation/1/'
for i in val0ImageNames:
  imgDir=trainDir+i
  imgchangeDir=changeDir + i
  my_file = Path(imgDir)
  if my_file.is_file():
      os.rename(imgDir, imgchangeDir)
# test folder
changeDir= 'content/split/test/1/'
for i in test0ImageNames:
  imgDir=trainDir+i
  imgchangeDir=changeDir + i
  my_file = Path(imgDir)
  if my_file.is_file():
      os.rename(imgDir, imgchangeDir)

In [0]:
# define training and test data directories
train_dir = 'content/split/train'
val_dir = 'content/split/validation'
test_dir = 'content/split/test'

# load and transform data using ImageFolder
data_transform = transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

train_data = datasets.ImageFolder(train_dir, transform=data_transform)
val_data = datasets.ImageFolder(val_dir, transform=data_transform)
test_data = datasets.ImageFolder(test_dir, transform=data_transform)

# Training AlexNet/CNN Stage 1 with Full Dataset 

In [0]:
import torchvision.models
alexnet = torchvision.models.alexnet(pretrained=True)

In [0]:
batch_size = 64
num_workers = 0

# create a dataloader
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
                                       num_workers=num_workers, shuffle=False)
# validation dataloader
val_data_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, 
                                       num_workers=num_workers, shuffle=False)
# testing dataloader
test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
                                       num_workers=num_workers, shuffle=False)

In [0]:
#Compute the AlexNet features
i = 0
for imgs, labels in train_data_loader:
    i = i + 1
    train_features = alexnet.features(imgs)
    torch.save(train_features, 'train_features_' + str(i) + '.pt')

i = 0
for imgs, labels in val_data_loader:
    i = i + 1
    val_features = alexnet.features(imgs)
    torch.save(val_features, 'val_features_' + str(i) + '.pt')

i = 0
for imgs, labels in test_data_loader:
    i = i + 1
    test_features = alexnet.features(imgs)
    torch.save(test_features, 'test_features_' + str(i) + '.pt')

In [0]:
#Re-load saved features
path = '/content/drive/My Drive/APS360 2019 September Team/Steel Defect Detection/Features'
features = os.listdir(path)
for feature in features:
    shutil.copy(path + feature, "/content/" + feature)

#Combine train_features back into 1 tensor for training set
train_features = torch.load("train_features_" + str(1) + ".pt")
for i in range(1, 2):
    train_features = torch.cat([train_features, torch.load("train_features_" + str(i+1) + ".pt") ], 0)
val_features = torch.load("val_features_" + str(1) + ".pt")
test_features = torch.load("test_features_" + str(1) + ".pt")

In [0]:
class alexNet(nn.Module):
      def __init__(self):
          super(alexNet, self).__init__()
          self.name = "alexNet_precompute"
          self.fc1 = nn.Linear(9216, 800)
          self.fc2 = nn.Linear(800, 300)
          self.fc3 = nn.Linear(300, 2)
 
      def forward(self, x):
          x = x.view(x.size(0), -1) #flatten the input
          x = F.relu(self.fc1(x))
          x = F.relu(self.fc2(x))
          x = self.fc3(x)
          x = x.squeeze(1)
          return x

In [0]:
def make_trainable_features(alexNetFeatures, image_data):
    x = [] 
    for i, img in enumerate(alexNetFeatures):
        x.append((img, image_data[i][1])) #append the image tensor and the label
    return x

In [0]:
#training
train_alexNet = make_trainable_features(train_features, train_data)
val_alexNet = make_trainable_features(val_features, val_data)
model = alexNet()
train(model, train_alexNet, val_alexNet, batch_size=batch_size, num_epochs=50, learning_rate = 0.0001, useGPU = True)

In [0]:
# testing 
get_accuracy(model, test_data_loader, useGPU=True)

NameError: ignored